In [2]:
import argparse
import redis
from urllib.parse import urlparse
import imageio

In [7]:
# Set up some vars
initialized_key = 'cats:initialized'

# Set up Redis connection
url = urlparse('redis://127.0.0.1:6379')
conn = redis.Redis(host=url.hostname, port=url.port)
if not conn.ping():
    raise Exception('Redis unavailable')

# Check if this Redis instance had already been initialized
initialized = conn.exists(initialized_key)
if initialized:
    print('Discovered evidence of a previous initialization - skipping.')
    exit(0)

# Load the RedisAI model
print('Loading model - ', end='')
with open('../model.pt', 'rb') as f:
    model = f.read()
    res = conn.execute_command(
        'AI.MODELSET', 
        'model', 
        'TORCH', 
        'CPU', 
        'INPUTS', 'input', 
        'OUTPUTS', 'pred', 
        model)
    print(res)

Loading model - b'OK'


In [11]:
# Load the gear
print('Loading gear - ', end='')
with open('gear_stream.py', 'rb') as f:
    gear = f.read()
    res = conn.execute_command(
        'RG.PYEXECUTE', gear,
        'REQUIREMENTS', 'imageio', 'python-twitter', 'opencv-python')
    print(res)

# Lastly, set a key that indicates initialization has been performed
print('Flag initialization as done - ', end='')

print(conn.set(initialized_key, 'miauw'))

Loading gear - 

ResponseError: ['Traceback (most recent call last):\n', '  File "<string>", line 6, in <module>\n', '  File "/var/opt/redislabs/modules/rg/python3_0.99.1/lib/python3.7/site-packages/cv2/__init__.py", line 5, in <module>\n    from .cv2 import *\n', 'ImportError: libgthread-2.0.so.0: cannot open shared object file: No such file or directory\n']